## Data
Being able to easily load and process data is a crucial task that can make any data science more pleasant. In this notebook, we will cover most common types often encountered in data science tasks, and we will be using this data throughout the rest of this tutorial.

In [178]:
#importing the packages that I did not have installed yet
import Pkg
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("XLSX")

#later, we will import data files in different formats. For this, we need to install the following ones:
#Pkg.add("JLD")
#Pkg.add("NPZ")
#Pkg.add("RData")
#Pkg.add("RCall")
#Pkg.add("MAT")

In [3]:
using BenchmarkTools
using DataFrames
using DelimitedFiles
using CSV
using XLSX
using Downloads

# 🗃️ Get some data
In Julia, it's pretty easy to dowload a file from the web using the `download` (https://docs.julialang.org/en/v1/stdlib/Downloads/) function. But also, you can use your favorite command line commad to download files by easily switching from Julia via the `;` key. Let's try both.

Note: `download` depends on external tools such as curl, wget or fetch. So you must have one of these.

In [5]:
P = Downloads.download("https://raw.githubusercontent.com/nassarhuda/easy_data/master/programming_languages.csv",
    "programming_languages.csv") #we download the file into our folder as programming_languages.csv & store it in the coding file as P

"programming_languages.csv"

Another way would be to use a shell command to get the same file.

# 📂 Read your data from text files.
The key question here is to load data from files such as `csv` files, `xlsx` files, or just raw text files. We will go over some Julia packages that will allow us to read such files very easily.

Let's start with the package `DelimitedFiles` which is in the standard library.

In [19]:
#=
readdlm(source, 
    delim::AbstractChar, 
    T::Type, 
    eol::AbstractChar; 
    header=false, 
    skipstart=0, 
    skipblanks=true, 
    use_mmap, 
    quotes=true, 
    dims, 
    comments=false, 
    comment_char='#')
=#
P,H = readdlm("programming_languages.csv",',';header=true); #header true means that the csv file has a header integrated

In [21]:
H #now we can access the headers again
# if we do not put header=true but header=false, printing H gives us the first cell, namely 1951

1×2 Matrix{AbstractString}:
 "year"  "language"

In [23]:
P #using P gives us the full matrix

73×2 Matrix{Any}:
 1951  "Regional Assembly Language"
 1952  "Autocode"
 1954  "IPL"
 1955  "FLOW-MATIC"
 1957  "FORTRAN"
 1957  "COMTRAN"
 1958  "LISP"
 1958  "ALGOL 58"
 1959  "FACT"
 1959  "COBOL"
 1959  "RPG"
 1962  "APL"
 1962  "Simula"
    ⋮  
 2003  "Scala"
 2005  "F#"
 2006  "PowerShell"
 2007  "Clojure"
 2009  "Go"
 2010  "Rust"
 2011  "Dart"
 2011  "Kotlin"
 2011  "Red"
 2011  "Elixir"
 2012  "Julia"
 2014  "Swift"

In [25]:
# To write to a text file, you can:
writedlm("programminglanguages_dlm.txt", P, '-')
# we are now pasting P into the text file using the delimiter "-"

A more powerful package to use here is the `CSV` package. By default, the CSV package imports the data to a DataFrame, which can have several advantages as we will see below.

In general,[`CSV.jl`](https://juliadata.github.io/CSV.jl/stable/) is the recommended way to load CSVs in Julia. Only use `DelimitedFiles` when you have a more complicated file where you want to specify several things.

In [27]:
C = CSV.read("programming_languages.csv", DataFrame);

In [41]:
 #we can use @show to print several outputs
@show typeof(C) #we want to know the type of C, which is a Dataframe
C[1:10,:] #and we want to print the first ten rows
# C.year #this gives us a vector of all the years in C
# [!,:year] #

typeof(C) = DataFrame


Row,year,language
,Int64,String31
1,1951,Regional Assembly Language
2,1952,Autocode
3,1954,IPL
4,1955,FLOW-MATIC
5,1957,FORTRAN
6,1957,COMTRAN
7,1958,LISP
8,1958,ALGOL 58
9,1959,FACT


In [43]:
@show typeof(P)
P[1:10,:]

typeof(P) = Matrix{Any}


10×2 Matrix{Any}:
 1951  "Regional Assembly Language"
 1952  "Autocode"
 1954  "IPL"
 1955  "FLOW-MATIC"
 1957  "FORTRAN"
 1957  "COMTRAN"
 1958  "LISP"
 1958  "ALGOL 58"
 1959  "FACT"
 1959  "COBOL"

In [45]:
names(C) #this gives us the names of the different columns

2-element Vector{String}:
 "year"
 "language"

In [59]:
#here we can go on to check some descriptive statistics
names(C) #we first check the names of the columns
C.year #we then proceed to look at the column year
C.language #we then move on to look at the column language
describe(C) #finally, we look at the descriptive statistics

#even though language is a string, we get an output

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,year,1982.99,1951,1986.0,2014,0,Int64
2,language,,ALGOL 58,,dBase III,0,String31


In [61]:
#let's compare both of the methods to read in CSVs
@btime P,H = readdlm("programming_languages.csv",',';header=true);
@btime C = CSV.read("programming_languages.csv", DataFrame);

  203.600 μs (328 allocations: 50.71 KiB)
  314.400 μs (472 allocations: 47.27 KiB)


In [63]:
# To write to a *.csv file using the CSV package
CSV.write("programminglanguages_CSV.csv", DataFrame(P, :auto))

"programminglanguages_CSV.csv"

Another type of files that we may often need to read is `XLSX` files. Let's try to read a new file.

In [80]:
T = XLSX.readdata("data/zillow_data_download_april2020.xlsx", #file name
    "Sale_counts_city", #sheet name
    "A1:F9" #cell range
    )

9×6 Matrix{Any}:
      "RegionID"  "RegionName"    …      "2008-03"      "2008-04"
  6181            "New York"             missing        missing
 12447            "Los Angeles"      1446           1705
 39051            "Houston"          2926           3121
 17426            "Chicago"          2910           3022
  6915            "San Antonio"   …  1479           1529
 13271            "Philadelphia"     1609           1795
 40326            "Phoenix"          1310           1519
 18959            "Las Vegas"        1618           1856

If you don't want to specify cell ranges... though this will take a little longer...

In [91]:
G = XLSX.readtable("data/zillow_data_download_april2020.xlsx","Sale_counts_city"); 
#G = DataFrame(XLSX.readtable("data/zillow_data_download_april2020.xlsx", "Sale_counts_city"))

And we can easily store this data in a DataFrame:

In [104]:
D = DataFrame(G) # equivalent to DataFrame(G[1],G[2])

Row,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,⋯
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,⋯
1,6181,New York,New York,1,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,2687,2821,2865,3303,3405,2990,3186,3721,3484,3237,3112,2757,2814,3740,2540,3461,3444,3414,3961,3857,3820,3694,3363,3472,3395,4201,3394,3481,3146,4188,3455,4352,4855,5362,4760,3958,4655,4201,3203,3740,3542,4212,3710,3756,4382,4007,4464,3395,4273,3863,3703,4035,3763,3450,4769,4514,4397,5099,4526,3874,4545,3741,3276,⋯
2,12447,Los Angeles,California,2,1446,1705,1795,1890,2072,1876,1981,2195,1579,2068,1523,1514,1976,2128,2060,2536,2538,2282,2456,2448,2107,2538,1816,1820,2382,2262,2552,2651,2238,2099,2107,1904,1912,2371,1834,1723,2391,2169,2137,2457,2170,2563,2197,2016,2049,2401,1849,1952,2452,2401,2726,2727,2623,2853,2295,2671,2495,2881,1945,1969,2667,2642,2932,2748,2843,3071,2434,2621,2464,2533,2040,1923,2453,2831,2854,2788,2940,2687,2538,2875,2177,2661,2074,1988,2590,2824,2834,3106,3191,2909,2951,2742,2216,2830,2142,2095,⋯
3,39051,Houston,Texas,3,2926,3121,3220,3405,3352,2962,2017,2595,1865,2472,1776,1918,2472,2333,2443,2918,2923,2632,2530,2681,2524,2605,1627,1875,2761,2791,2988,3207,2391,2372,2158,2034,1968,2181,1960,1762,2707,2439,2758,2760,2541,2947,2387,2244,2017,2513,2014,2059,2655,2375,3164,2930,3290,2790,2641,2698,2672,2504,2288,2510,2922,3736,3880,3479,3859,3702,3335,3187,2750,3141,2492,2219,3210,3324,3522,3574,4167,2342,3578,3546,2751,3402,2491,2426,3228,3279,3394,4006,3855,3398,3341,3083,2373,3255,2372,2633,⋯
4,17426,Chicago,Illinois,4,2910,3022,2937,3224,3464,3371,2979,2869,1915,2110,1897,1618,2284,2520,2366,2938,3015,2706,2866,3235,2937,2776,2445,1913,2669,3208,3186,3506,3203,2744,2320,2095,1877,2233,2111,1792,2457,2149,2438,2988,2479,2998,2391,2255,2119,2296,2113,1657,2265,2311,2845,3458,2907,3610,2367,2423,2138,2021,2261,1743,2029,2813,2996,3022,3974,3622,2796,3734,2569,2682,2489,2074,2305,3540,3235,3474,3268,2896,3707,3516,2306,3059,2161,1945,2712,3347,3470,4084,4336,4612,4149,3832,2777,3150,2649,2534,⋯
5,6915,San Antonio,Texas,5,1479,1529,1582,1761,1678,1468,1480,1475,949,1178,856,1016,1283,1221,1339,1777,1752,1484,1460,1444,1380,1377,810,1111,1477,1651,1669,1796,1347,1333,1245,1166,1086,1148,962,960,1372,1237,1430,1539,1378,1435,1262,1160,1103,1193,1063,1024,1361,1389,1588,1643,1661,1690,1343,1521,1290,1388,1170,1270,1552,1790,1955,1925,2027,1992,1694,1631,1470,1587,1299,1232,1742,1729,1944,2029,2066,1928,2037,1846,1454,1797,1282,1405,2015,1897,2004,2325,2433,2208,2087,1948,1516,1931,1430,1706,⋯
6,13271,Philadelphia,Pennsylvania,6,1609,1795,1709,1914,1984,1739,1635,1598,1169,1414,1152,1029,1199,1368,1412,1586,1853,1464,1590,1679,1586,1

Let us now learn how to merge two data sets

In [119]:
# we manually create two different data sets
foods = ["apple", "cucumber", "tomato", "banana"]
calories = [105,47,22,105]
prices = [0.85,1.6,0.8,0.6,]
@show dataframe_calories = DataFrame(item=foods,calories=calories) #just want to check both dfs by using @show
dataframe_prices = DataFrame(item=foods,price=prices)

dataframe_calories = DataFrame(item = foods, calories = calories) = 4×2 DataFrame
 Row │ item      calories
     │ String    Int64
─────┼────────────────────
   1 │ apple          105
   2 │ cucumber        47
   3 │ tomato          22
   4 │ banana         105


Row,item,price
,String,Float64
1,apple,0.85
2,cucumber,1.6
3,tomato,0.8
4,banana,0.6


In [121]:
# we now merge both data frames on the item column
DF = innerjoin(dataframe_calories,dataframe_prices,on=:item)

Row,item,calories,price
,String,Int64,Float64
1,apple,105,0.85
2,cucumber,47,1.6
3,tomato,22,0.8
4,banana,105,0.6


In [123]:
# we can also use the DataFrame constructor on a Matrix
DataFrame(T, :auto)

Row,x1,x2,x3,x4,x5,x6
,Any,Any,Any,Any,Any,Any
1,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04
2,6181,New York,New York,1,missing,missing
3,12447,Los Angeles,California,2,1446,1705
4,39051,Houston,Texas,3,2926,3121
5,17426,Chicago,Illinois,4,2910,3022
6,6915,San Antonio,Texas,5,1479,1529
7,13271,Philadelphia,Pennsylvania,6,1609,1795
8,40326,Phoenix,Arizona,7,1310,1519
9,18959,Las Vegas,Nevada,8,1618,1856


## ⬇️ Importing your data

Often, the data you want to import is not stored in plain text, and you might want to import different kinds of types. Here we will go over importing `jld`, `npz`, `rda`, and `mat` files. Hopefully, these four will capture the types from four common programming languages used in Data Science (Julia, Python, R, Matlab).

We will use a toy example here of a very small matrix. But the same syntax will hold for bigger files.

```
4×5 Array{Int64,2}:
 2  1446  1705  1795  1890
 3  2926  3121  3220  3405
 4  2910  3022  2937  3224
 5  1479  1529  1582  1761
 ```

### Julia

In [150]:
using JLD
jld_data = JLD.load("data/mytempdata.jld")
save("mywrite.jld", "A", jld_data) #we have to store the data with a key, we are using key "A" now.

@show jld_data;

jld_data = Dict{String, Any}("tempdata" => [2 1446 1705 1795 1890; 3 2926 3121 3220 3405; 4 2910 3022 2937 3224; 5 1479 1529 1582 1761])


In [152]:
#let's load the data again and access it via the key "A"
loaded = JLD.load("mywrite.jld") 
loaded["A"]

Dict{String, Any} with 1 entry:
  "tempdata" => [2 1446 … 1795 1890; 3 2926 … 3220 3405; 4 2910 … 2937 3224; 5 …

### Python

In [158]:
using NPZ
npz_data = npzread("data/mytempdata.npz")
npzwrite("mywrite.npz", npz_data)

@show npz_data;

npz_data = [2 1446 1705 1795 1890; 3 2926 3121 3220 3405; 4 2910 3022 2937 3224; 5 1479 1529 1582 1761]


### R

In [172]:
using RData
R_data = RData.load("data/mytempdata.rda")
# We'll need RCall to save here. https://github.com/JuliaData/RData.jl/issues/56
using RCall
@rput R_data
R"save(R_data, file=\"mywrite.rda\")"

[ Info: Precompiling RCall [6f49c342-dc21-5d91-9882-a32aef131414] 


RObject{NilSxp}
NULL


### MATLAB

In [180]:
using MAT
Matlab_data = matread("data/mytempdata.mat")
matwrite("mywrite.mat",Matlab_data)

In [182]:
@show typeof(jld_data)
@show typeof(npz_data)
@show typeof(R_data)
@show typeof(Matlab_data)
;

typeof(jld_data) = Dict{String, Any}
typeof(npz_data) = Matrix{Int64}
typeof(R_data) = Dict{String, Any}
typeof(Matlab_data) = Dict{String, Any}


# 🔢 Time to process the data from Julia
We will mainly cover `Matrix` (or `Vector`), `DataFrame`s, and `dict`s (or dictionaries). Let's bring back our programming languages dataset and start playing it the matrix it's stored in.

In [184]:
P

73×2 Matrix{Any}:
 1951  "Regional Assembly Language"
 1952  "Autocode"
 1954  "IPL"
 1955  "FLOW-MATIC"
 1957  "FORTRAN"
 1957  "COMTRAN"
 1958  "LISP"
 1958  "ALGOL 58"
 1959  "FACT"
 1959  "COBOL"
 1959  "RPG"
 1962  "APL"
 1962  "Simula"
    ⋮  
 2003  "Scala"
 2005  "F#"
 2006  "PowerShell"
 2007  "Clojure"
 2009  "Go"
 2010  "Rust"
 2011  "Dart"
 2011  "Kotlin"
 2011  "Red"
 2011  "Elixir"
 2012  "Julia"
 2014  "Swift"

Here are some quick questions we might want to ask about this simple data.
- Which year was was a given language invented?
- How many languages were created in a given year?

In [186]:
# Q1: Which year was was a given language invented?
function year_created(P,language::String)
    loc = findfirst(P[:,2] .== language)
    return P[loc,1]
end
year_created(P,"Julia")

2012

In [192]:
year_created(P,"W") #W does not exist

LoadError: ArgumentError: invalid index: nothing of type Nothing

In [194]:
function year_created_handle_error(P,language::String)
    loc = findfirst(P[:,2] .== language)
    !isnothing(loc) && return P[loc,1]
    error("Error: Language not found.") #Let's add an error message now
end
year_created_handle_error(P,"W")

LoadError: Error: Language not found.

In [196]:
# Q2: How many languages were created in a given year?
function how_many_per_year(P,year::Int64)
    year_count = length(findall(P[:,1].==year))
    return year_count
end
how_many_per_year(P,2011)

4

Now let's try to store this data in a DataFrame...

In [198]:
P_df = C #DataFrame(year = P[:,1], language = P[:,2]) # or DataFrame(P)

Row,year,language
,Int64,String31
1,1951,Regional Assembly Language
2,1952,Autocode
3,1954,IPL
4,1955,FLOW-MATIC
5,1957,FORTRAN
6,1957,COMTRAN
7,1958,LISP
8,1958,ALGOL 58
9,1959,FACT


In [35]:
# Even better, since we know the types of each column, we can create the DataFrame as follows:
# P_df = DataFrame(year = Int.(P[:,1]), language = string.(P[:,2]))

And now let's answer the same questions we just answered...

In [200]:
# Q1: Which year was was a given language invented?
# it's a little more intuitive and you don't need to remember the column ids
function year_created(P_df,language::String)
    loc = findfirst(P_df.language .== language)
    return P_df.year[loc]
end
year_created(P_df,"Julia")

2012

In [202]:
year_created(P_df,"W")

LoadError: ArgumentError: invalid index: nothing of type Nothing

In [204]:
function year_created_handle_error(P_df,language::String)
    loc = findfirst(P_df.language .== language)
    !isnothing(loc) && return P_df.year[loc]
    error("Error: Language not found.") #again, here we handle the error
end
year_created_handle_error(P_df,"W")

LoadError: Error: Language not found.

In [206]:
# Q2: How many languages were created in a given year?
function how_many_per_year(P_df,year::Int64)
    year_count = length(findall(P_df.year.==year))
    return year_count
end
how_many_per_year(P_df,2011)

4

Next, we'll use dictionaries. A quick way to create a dictionary is with the `Dict()` command. But this creates a dictionary without types. Here, we will specify the types of this dictionary.

In [208]:
# A quick example to show how to build a dictionary
Dict([("A", 1), ("B", 2),(1,[1,2])])

Dict{Any, Any} with 3 entries:
  "B" => 2
  "A" => 1
  1   => [1, 2]

In [212]:
P_dictionary = Dict{Integer,Vector{String}}() #here we specify the type of variables for the dictionary

Dict{Integer, Vector{String}}()

In [42]:
#P_dictionary[67] = ["julia","programming"]

In [214]:
# this is not going to work.
P_dictionary["julia"] = 7

LoadError: MethodError: [0mCannot `convert` an object of type [92mString[39m[0m to an object of type [91mInteger[39m
The function `convert` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::EzXML.NodeType[39m) where T<:Integer
[0m[90m   @[39m [33mEzXML[39m [90mC:\Users\roell\.julia\packages\EzXML\qbIRq\src\[39m[90m[4mnode.jl:36[24m[39m
[0m  convert(::Type{T}, [91m::EzXML.ReaderType[39m) where T<:Integer
[0m[90m   @[39m [33mEzXML[39m [90mC:\Users\roell\.julia\packages\EzXML\qbIRq\src\[39m[90m[4mstreamreader.jl:74[24m[39m
[0m  convert(::Type{S}, [91m::CategoricalArrays.CategoricalValue[39m) where S<:Union{AbstractChar, AbstractString, Number}
[0m[90m   @[39m [36mCategoricalArrays[39m [90mC:\Users\roell\.julia\packages\CategoricalArrays\0yLZN\src\[39m[90m[4mvalue.jl:92[24m[39m
[0m  ...


Now, let's populate the dictionary with years as keys and vectors that hold all the programming languages created in each year as their values. Even though this looks like more work, we often need to do it just once.

In [216]:
dict = Dict{Integer,Vector{String}}()
for i = 1:size(P,1)
    year,lang = P[i,:]
    if year in keys(dict)
        dict[year] = push!(dict[year],lang) 
        # note that push! is not our favorite thing to do in Julia, 
        # but we're focusing on correctness rather than speed here
    else
        dict[year] = [lang]
    end
end

In [218]:
# Though a smarter way to do this is:
curyear = P_df.year[1]
P_dictionary[curyear] = [P_df.language[1]]
for (i,nextyear) in enumerate(P_df.year[2:end])
    if nextyear == curyear
        #same key
        P_dictionary[curyear] = push!(P_dictionary[curyear],P_df.language[i+1])
        # note that push! is not our favorite thing to do in Julia, 
        # but we're focusing on correctness rather than speed here
    else
        curyear = nextyear
        P_dictionary[curyear] = [P_df.language[i+1]]
    end
end

In [220]:
length(keys(P_dictionary))

45

In [222]:
length(unique(P[:,1]))

45

In [224]:
# Q1: Which year was was a given language invented?
# now instead of looking in one long vector, we will look in many small vectors
function year_created(P_dictionary,language::String)
    keys_vec = collect(keys(P_dictionary))
    lookup = map(keyid -> findfirst(P_dictionary[keyid].==language),keys_vec)
    # now the lookup vector has `nothing` or a numeric value. We want to find the index of the numeric value.
    return keys_vec[findfirst((!isnothing).(lookup))]
end
year_created(P_dictionary,"Julia")

2012

In [226]:
# Q2: How many languages were created in a given year?
how_many_per_year(P_dictionary,year::Int64) = length(P_dictionary[year])
how_many_per_year(P_dictionary,2011)

4

# 📝 A note about missing data

In [228]:
# assume there were missing values in our dataframe
P[1,1] = missing
P_df = DataFrame(year = P[:,1], language = P[:,2])

Row,year,language
,Any,Any
1,missing,Regional Assembly Language
2,1952,Autocode
3,1954,IPL
4,1955,FLOW-MATIC
5,1957,FORTRAN
6,1957,COMTRAN
7,1958,LISP
8,1958,ALGOL 58
9,1959,FACT


In [232]:
dropmissing(P_df) #we can easily drop missing data using dropmissing

Row,year,language
,Any,Any
1,1952,Autocode
2,1954,IPL
3,1955,FLOW-MATIC
4,1957,FORTRAN
5,1957,COMTRAN
6,1958,LISP
7,1958,ALGOL 58
8,1959,FACT
9,1959,COBOL


# Finally...
After finishing this notebook, you should be able to:
- [ ] dowload a data file from the web given a url
- [ ] load data from a file from a text file via DelimitedFiles or CSV
- [ ] write your data to a text file or csv file
- [ ] load data from file types xlsx, jld, npz, mat, rda
- [ ] write your data to an xlsx file, jld, npz, mat, rda
- [ ] store data in a 2D array (`Matrix`), or `DataFrame` or `Dict`
- [ ] write functions to perform basic lookups on `Matrix`, `DataFrame`, and `Dict` types
- [ ] use some of the basic functions on `DataFrame`s such as: `dropmissing`, `describe`, `combine(groupby)`, and `innerjoin`

# 🥳 One cool finding

Julia was created in 2012